In [ ]:
#template Promotion Graph
import pandas as pd
import numpy as np 
from datetime import datetime
import os

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import json
import re
dates = ['29']
df_fe = pd.DataFrame()
df_cashier = pd.DataFrame()

for date in dates:
    df_fe = pd.concat([df_fe, pd.read_csv('/Users/lap13729/Documents/Data/From Jupyter/Cashier full/Aug{}ZPI_promotionScreen.csv'.format(date))])
    df_cashier = pd.concat([df_cashier, pd.read_csv('/Users/lap13729/Documents/Data/From Jupyter/Cashier_ZDS/Aug/Aug{}_CashierLog.csv'.format(date))])

df_fe.at[0,'metadata']
df_cashier = df_cashier[df_cashier['source_type'] == 'ZPI']
df_fe = df_fe[df_fe['eventId'].isin(['02.1104.006','02.1106.006','02.1115.000','02.1104.053'])]
df_fe['metadata'] = df_fe['metadata'].apply(json.loads)

df_cashier['metadata'] = df_cashier['metadata'].apply(json.loads)

list_fe = ['number_of_available_promotion','promotion_error_code']
for i in list_fe:
    df_fe[i] = df_fe['metadata'].map(lambda x: x[i] if i in x else np.nan)

list_cashier = ['total_promotion','best_promotion_applied']
for j in list_cashier:
    df_cashier[j] = df_cashier['metadata'].map(lambda x: x[j] if j in x else np.nan)
viewPromoList = df_fe[
    df_fe['eventId'].isin(['02.1104.006','02.1106.006'])
]

applyPromo = df_fe[
    df_fe['eventId'] == '02.1115.000'
]

viewPromoDetail = df_fe[
    df_fe['eventId'] == '02.1104.053'
] 

temp_cashier = df_cashier[
    (df_cashier['source_type'] == 'ZPI')
    & (~df_cashier['domain'].isin(['Funding Channel','IBFT','Transfer','Withdraw']))
]
viewPromoList.head()
#Filter View Promo or not?
temp_cashier['viewPromo?'] = 0
temp_cashier['viewPromo?'][
    (temp_cashier['trackingSessionId'].isin(viewPromoList['trackingSessionId'].unique().tolist()))
] = 1

#Filter have valid promo when open list or not
haveValid = viewPromoList[viewPromoList['number_of_available_promotion'].astype('int') > 0]

temp_cashier['haveValid?'] = 0
temp_cashier['haveValid?'][
    temp_cashier['trackingSessionId'].isin(haveValid['trackingSessionId'].unique().tolist())
] = 1

#Filter apply promo or not
temp_cashier['applyPromo?'] = 0
temp_cashier['applyPromo?'][
    temp_cashier['trackingSessionId'].isin(applyPromo['trackingSessionId'].unique().tolist())
] = 1
#Filter apply promo success or not
applySuccess = applyPromo[applyPromo['promotion_error_code'] == '']
temp_cashier['applySuccess?'] = 0
temp_cashier['applySuccess?'][
    temp_cashier['trackingSessionId'].isin(applySuccess['trackingSessionId'].unique().tolist())
] = 1
#Filter view promotion detail
temp_cashier['viewDetail?'] = 0
temp_cashier['viewDetail?'][
    temp_cashier['trackingSessionId'].isin(viewPromoDetail['trackingSessionId'].unique().tolist())
] = 1
 # done filtering!

#make a table 
dr = ['Load Cashier',
    'Not Auto-applied',
    'Auto-applied',
    'Not Auto-applied - Have Promo',
    'Not Auto-applied - Not Have Promo',
    'Auto-applied - View Promo List',
    'Auto-applied - Not View Promo List',
    'Not Auto-applied - Have Promo - Not View Promo List',
    'Not Auto-applied - Have Promo - View Promo List',
    'Not Auto-applied - Not Have Promo - Not View Promo List',
    'Not Auto-applied - Not Have Promo - View Promo List',
    'Have Valid Promo in List',
    'Not Have Valid Promo in List',
    'Apply Promo',
    'Not Apply Promo',
    'Apply Success',
    'Apply Failed',
    'Not Apply - View Detail',
    'Not Apply - Not View Detail',
    'Not Have Valid Promo in List - View Detail',
    'Not Have Valid Promo in List - Not View Detail',
    ]

dr = pd.DataFrame(dr)
dr
total_loads = temp_cashier.shape[0]
df = temp_cashier
df['best_promotion_applied'].unique()
res = pd.DataFrame(
    columns = ['name',
                'Load Cashier',
                'Not Auto-applied',
                'Auto-applied',
                'Not Auto-applied - Have Promo',
                'Not Auto-applied - Not Have Promo',
                'Auto-applied - View Promo List',
                'Auto-applied - Not View Promo List',
                'Not Auto-applied - Have Promo - Not View Promo List',
                'Not Auto-applied - Have Promo - View Promo List',
                'Not Auto-applied - Not Have Promo - Not View Promo List',
                'Not Auto-applied - Not Have Promo - View Promo List',
                'Have Valid Promo in List',
                'Not Have Valid Promo in List',
                'Apply Promo',
                'Not Apply Promo',
                'Apply Success',
                'Apply Failed',
                'Not Apply - View Detail',
                'Not Apply - Not View Detail',
                'Not Have Valid Promo in List - View Detail',
                'Not Have Valid Promo in List - Not View Detail',
                ]
)
res['name'] = ['load','submit','success','distribution','submission rate','be sr','e2e sr']

for i, column in enumerate(['Load Cashier',
                            'Not Auto-applied',
                            'Auto-applied',
                            'Not Auto-applied - Have Promo',
                            'Not Auto-applied - Not Have Promo',
                            'Auto-applied - View Promo List',
                            'Auto-applied - Not View Promo List',
                            'Not Auto-applied - Have Promo - Not View Promo List',
                            'Not Auto-applied - Have Promo - View Promo List',
                            'Not Auto-applied - Not Have Promo - Not View Promo List',
                            'Not Auto-applied - Not Have Promo - View Promo List',
                            'Have Valid Promo in List',
                            'Not Have Valid Promo in List',
                            'Apply Promo',
                            'Not Apply Promo',
                            'Apply Success',
                            'Apply Failed',
                            'Not Apply - View Detail',
                            'Not Apply - Not View Detail',
                            'Not Have Valid Promo in List - View Detail',
                            'Not Have Valid Promo in List - Not View Detail',
                            ]):
    if i == 0:
        df = temp_cashier
    elif i == 1:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
        ]
    elif i == 2:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
        ]
    elif i == 3:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
        ]
    elif i == 4:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)

        ]
    elif i == 5:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 6:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] == True)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 7:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 8:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 9:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 0)
        ]
    elif i == 10:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') == 0)
            & (temp_cashier['viewPromo?'] == 1)
        ]
    elif i == 11:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
        ]
    elif i == 12:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
        ]
    elif i == 13:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
        ]
    elif i == 14:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
        ]
    elif i == 15:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
            & (temp_cashier['applySuccess?'] == 1)
        ]
    elif i == 16:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 1)
            & (temp_cashier['applySuccess?'] == 0)
        ]
    elif i == 17:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
            & (temp_cashier['viewDetail?'] == 1)
        ]
    elif i == 18:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 1)
            & (temp_cashier['applyPromo?'] == 0)
            & (temp_cashier['viewDetail?'] == 0)
        ]
    elif i == 19:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 1)
        ]
    elif i == 20:
        df = temp_cashier[
            (temp_cashier['best_promotion_applied'] != True)
            & (temp_cashier['total_promotion'].astype('int') > 0)
            & (temp_cashier['viewPromo?'] == 1)
            & (temp_cashier['haveValid?'] == 0)
            & (temp_cashier['viewDetail?'] == 0)
        ]
    
    #index
    load = len(df)
    submit = len(df[df['is_submitted'] == 1])
    success = len(df[df['is_success'] == 1])
    distribution = load/total_loads
    submission_rate = submit/load
    be_sr = success/submit
    e2e_sr = success/load

    #table
    res[column] =[
        load,
        submit,
        success,
        distribution,
        submission_rate,
        be_sr,
        e2e_sr,
    ]


res